In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

import holoviews as hv
from holoviews import opts, dim
import panel as pn
import hvplot.pandas  # noqa: API import
import hvplot.xarray
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
import datashader.transfer_functions as tf
import datashader as ds

import pickle
import molecule_counting_v2 as molco

from scipy import stats
import importlib

In [ ]:
hv.extension("bokeh")

# Load data

In [ ]:
# with open('190311photobleaching.pickle', 'rb') as f:
# with open('all_data.pickle', 'rb') as f:
# with open('190313photobleaching_noflatcorr.pickle', 'rb') as f:
# with open('190326photobleaching_flatcorr_fluoronly.pickle', 'rb') as f:
with open(
    "/n/groups/paulsson/nao5/molecule-counting/190604_mothermachine.pickle", "rb"
) as f:
    d = pickle.load(f)
list(d.keys())

In [ ]:
import segmentation

%output size=200
mkate100_dir = "/n/scratch2/jqs1/190604/mkate_mcherry_100ms_100pct.nd2"

# fov = 2
# tabs = {}
tabs = pn.Tabs()
for fov in range(len(d[mkate100_dir])):
    seg_frame = np.asarray(d[mkate100_dir][fov]["segmentation_frame"])
    label_frame = segmentation.permute_labels(
        np.asarray(d[mkate100_dir][fov]["labels"])
    )
    #     label_frame = np.asarray(d[mkate100_dir][fov]['labels'])

    x_coord = np.arange(seg_frame.shape[0])
    y_coord = np.arange(seg_frame.shape[1])
    seg_frame = hv.Dataset((x_coord, y_coord, seg_frame), ["x", "y"], "Fluor")
    label_frame = hv.Dataset((x_coord, y_coord, label_frame), ["x", "y"], "label")
    img = datashade(seg_frame.to(hv.Image, ["x", "y"], "Fluor"), cmap=cm.jet).opts(
        bgcolor="black", active_tools=["wheel_zoom"]
    )
    labels = datashade(label_frame.to(hv.Image, ["x", "y"], "label"), cmap=cm.hot).opts(
        bgcolor="black", active_tools=["wheel_zoom"]
    )

    #     img = seg_frame.to(hv.Image,['x','y'],'Fluor')
    #     labels = label_frame.to(hv.Image,['x','y'],'label')
    combo = img + labels
    tabs.append((f"FOV: {fov}", combo))
# measurements, regionprops, labels, img = data[ '/n/scratch2/jqs1/190604/GFP_FITC_100ms_60pct.nd2'][4]
# t = pn.Tabs(('FOV 0',tabs[0]), ('FOV 1',tabs[1]))
tabs.clone(closable=True)

In [ ]:
importlib.reload(molco)

mvenus_dir = "/n/scratch2/jqs1/190604/mVenus_YFP_100ms_10pct.nd2"
mkate50_dir = "/n/scratch2/jqs1/190604/mkate_mcherry_100ms_50pct.nd2"
mkate100_dir = "/n/scratch2/jqs1/190604/mkate_mcherry_100ms_100pct.nd2"
gfp_dir = "/n/scratch2/jqs1/190604/GFP_FITC_100ms_60pct.nd2"

mkate50_bad_fov = [0, 3, 5, 6, 7]
mkate100_bad_fov = [1, 8, 10, 17, 19, 20]
mvenus_bad_fov = [0]
# mvenus_bad_fov = [0,8,9,10,11,12,13,14,15,16,17]
gfp_bad_fov = [8, 10, 11, 12, 13, 14, 15, 16, 17]

# data = molco.process_df(d,mkate50_dir,mkate50_bad_fov,6,1.8,.9)
data, meta_data = molco.process_df(d, mkate100_dir, mkate100_bad_fov, 6, 1.3, 0.90)
# data = molco.process_df(d,mvenus_dir,mvenus_bad_fov,6,1.5,.9)
# data = molco.process_df(d,gfp_dir,gfp_bad_fov,6,1.2,.95)
N = data["cell_id"].max()

In [ ]:
%output size=150


def line(ds):
    return ds.dframe().hvplot.line(
        x="time", y="normed", by="cell_id", line_width=6, alpha=0.7
    )


def sample(ds, n_samp=100):
    samp = list(np.random.choice(N, n_samp))
    return ds.select(cell_id=samp)


def good(ds, meta_ds, thresh=1):
    good_cells = list(meta_ds.select(dy=(0, thresh)).dframe().cell_id)
    good_ds = ds.select(cell_id=good_cells)

    return good_ds


def bad(ds, meta_ds, thresh=1):
    bad_cells = list(meta_ds.select(dy=(thresh, 1000)).dframe().cell_id)
    bad_ds = ds.select(cell_id=bad_cells)
    return bad_ds


sample_slider = pn.widgets.IntSlider(
    name="Number of Samples", value=50, start=1, end=500
)

ds_small = data.apply(sample, n_samp=sample_slider.param.value)

thresh_slider = pn.widgets.FloatSlider(
    name="Filter Threshold", value=1.2, start=1, end=3, step=0.01
)
good_ds = ds_small.apply(good, meta_ds=meta_data, thresh=thresh_slider.param.value)
bad_ds = ds_small.apply(bad, meta_ds=meta_data, thresh=thresh_slider.param.value)

dynamic_line = pn.panel(
    datashade(ds_small.apply(line)).opts(
        width=500, height=400, logy=True, tools=["hover"], ylim=(0.01, 1)
    )
)

# good_curves = pn.panel(datashade(good_ds.apply(line)).opts(width=400,height=400,tools=['hover'],ylim=(0.01,1)))
# bad_curves = pn.panel(datashade(bad_ds.apply(line)).opts(width=400,height=400,tools=['hover'],ylim=(0.01,1)))
good_curves = good_ds.apply(line)
bad_curves = bad_ds.apply(line)
# r = datashade(good_curves + bad_curves)
c1 = pn.Column(
    sample_slider,
    thresh_slider,
    datashade((ds_small.apply(line) + good_curves * bad_curves).cols(1)),
)
# pn.Row(good_curves, bad_curves)
# c2 = pn.Column(thresh_slider, r)
# dash = pn.Column(c1,c2)
# dash
c1

In [ ]:
%%output size=250
traces = data.iloc[:, :950].values
idxs = np.random.permutation(traces.shape[0])
downsample = 100  # set to 1 to show all traces (instead odata.k.hist()f 10%); this will make your browser slow
curves = [
    {"x": np.arange(traces.shape[1]), "y": traces[i], "i": idxs[i]}
    for i in range(traces.shape[0] // downsample)
]
hv.Contours(curves, vdims=["i"]).options(color_index="i", cmap="Category20", logy=True)

In [ ]:
def line(ds):
    return ds.dframe().hvplot.line(
        x="time", y="Fluorescence", by="cell_id", line_width=6
    )


def sample(ds, n_samp=100):
    samp = list(np.random.choice(N, n_samp))
    return ds.select(cell_id=samp)


sample_slider = pn.widgets.IntSlider(
    name="Number of Samples", value=50, start=1, end=500
)

ds_small = ds.apply(sample, n_samp=sample_slider.param.value)
# print(type(ds_small))
dynamic_line = pn.panel(
    datashade(ds_small.apply(line)).opts(
        width=700, height=400, logy=True, tools=["hover"]
    )
)


r = pn.Column(sample_slider, dynamic_line)
r

In [ ]:
import segmentation

%output size=200
fov = 2
img = np.asarray(d[mkate100_dir][fov]["segmentation_frame"])
labels = np.asarray(d[mkate100_dir][fov]["labels"])
# measurements, regionprops, labels, img = data[ '/n/scratch2/jqs1/190604/GFP_FITC_100ms_60pct.nd2'][4]
labels = segmentation.permute_labels(labels)
hv.Image(img / img.max()).options(cmap="hot") + hv.Image(labels)

In [ ]:
importlib.reload(molco)
# prop_dict = {'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#               0: [2000, 4500], t_end-1: [500,1000],
#              'k': [.003,.0045]}
# prop_dict = {
#             'centroid_y': [0, 2000],
#              'centroid_x': [0, 2000],
#              'area': [data.area.median() - 2*data.area.std(), data.area.median() + 2*data.area.std()],
#              0: [500, 6000],'k': [.002,.01]}
prop_dict = {
    "centroid_y": [400, 1600],
    "centroid_x": [700, 1800],
    "area": [
        data.area.median() - 2 * data.area.std(),
        data.area.median() + 2 * data.area.std(),
    ],
    0: [500, 3000],
    "k": [0.001, 0.02],
}  # , 1150:[0,350]}
df = molco.filter_df(data, prop_dict)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.hist(df.k, bins=30)
ax2.hist(df[0], bins=30)

q = 1

f_predicted, fbar, p, mu, sigma2, y = molco.fluct_plot(df, n=3)

# plt.figure(figsize=(12,8))
# yn = y.apply(lambda col: col/col.iloc[0],axis=1)
# plt.semilogy(yn.T.values[:,::20])
# plt.ylim(bottom=1e-3,top=1.1)
# plt.xlim(left=0)

plt.figure(figsize=(10, 8))
yn = y.apply(lambda col: col / col.iloc[0], axis=1)
tp = np.arange(yn.shape[1]) / 10
plt.semilogy(tp, yn.T.values[:, ::10], linewidth=3)
plt.semilogy(tp, mu / mu[0], "b", linewidth=7, alpha=0.7, label=r"$\mu$")
plt.xlabel("Time", fontsize=28)
plt.ylabel("Fluorescence", fontsize=28)
plt.legend(fontsize=20)
plt.ylim(bottom=1e-3, top=1.1)
plt.xlim(left=0)

plt.figure(figsize=(12, 8))
molco.kmaps(df)

In [ ]:
plt.figure(figsize=(12, 10))
plt.rc("xtick", labelsize=24)
plt.rc("ytick", labelsize=24)
t = 0.1 * np.arange(y.shape[1])
plt.semilogy(t, y.T.values[:, ::20])
plt.ylim(bottom=1e1, top=6e3)
plt.xlim(left=0)
plt.xlabel("Time (seconds)", fontsize=28)
plt.ylabel("Fluorescence (A.U.)", fontsize=28)
# plt.title('Photobleach Time Series')